## Description
Since we only play from time to time I thought that regardless of global winrate and simulations, one thing that may impact our performance is the recent changes in the champions statistics or items that could impact specific champions.

In [15]:
import pandas as pd
from scipy.stats import ttest_ind
import plotly.graph_objects as go

In [8]:
winrate_w12 = {
    'champion': ['Maokai', 'Ornn', 'Elise', 'Graves'],
    'winrate': [54.5, 51.2, 48.3, 52.7]
}
winrate_w34 = {
    'champion': ['Maokai', 'Ornn', 'Elise', 'Graves'],
    'winrate': [52.5, 52.1, 47.8, 50.3]
}

df_w12 = pd.DataFrame(winrate_w12)
df_w34 = pd.DataFrame(winrate_w34)


In [10]:
df_combined = pd.merge(df_w12, df_w34, on='champion', suffixes=('_w12', '_w34'))

In [11]:
results = []

for champ in df_combined['champion'].unique():
    winrate_w12 = df_combined[df_combined['champion'] == champ]['winrate_w12']
    winrate_w34 = df_combined[df_combined['champion'] == champ]['winrate_w34']
    
    t_stat, p_value = ttest_ind(winrate_w12, winrate_w34, equal_var=False)
    results.append({'champion': champ, 't_stat': t_stat, 'p_value': p_value})

results_df = pd.DataFrame(results)

/var/folders/9m/rglxw5bd15g0669_6zc63m000000gn/T/ipykernel_76014/839421858.py:7: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = ttest_ind(winrate_w12, winrate_w34, equal_var=False)
/Users/amberlights/Repositories/simple-lol-stats/.venv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/Users/amberlights/Repositories/simple-lol-stats/.venv/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1214: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


In [12]:
def interpret_p_value(p):
    if p < 0.001:
        return 'Significant at 99.9%'
    elif p < 0.01:
        return 'Significant at 99%'
    elif p < 0.05:
        return 'Significant at 95%'
    else:
        return 'Not significant'

results_df['interpretation'] = results_df['p_value'].apply(interpret_p_value)

In [13]:
print(results_df[['champion', 't_stat', 'p_value', 'interpretation']])

  champion  t_stat  p_value   interpretation
0   Maokai     NaN      NaN  Not significant
1     Ornn     NaN      NaN  Not significant
2    Elise     NaN      NaN  Not significant
3   Graves     NaN      NaN  Not significant


In [17]:
df_combined['winrate_diff'] = df_combined['winrate_w12'] - df_combined['winrate_w34']

In [29]:
def create_horizontal_bar_chart(df, role):
    fig = go.Figure()

    # Add bars for win rates in weeks 34
    fig.add_trace(go.Bar(
        x=-df['winrate_w34'],  # Negative values for left side
        y=df['champion'],
        name='Weeks 34',
        orientation='h',
        marker_color='#D4AF37',
        hovertemplate='%{x}%'
    ))

    # Add bars for win rates in weeks 12
    fig.add_trace(go.Bar(
        x=df['winrate_w12'],
        y=df['champion'],
        name='Weeks 12',
        orientation='h',
        marker_color='#D4AF37',
        hovertemplate='%{x}%'
    ))

    # Add bars for winrate differences
    for index, row in df.iterrows():
        if row['winrate_diff'] > 0:
            fig.add_trace(go.Bar(
                x=[row['winrate_diff']],
                y=[row['champion']],
                name='Increase',
                orientation='h',
                marker_color='blue',
                base=row['winrate_w34'],
                hovertemplate='%{x}%'
            ))
        else:
            fig.add_trace(go.Bar(
                x=[-row['winrate_diff']],
                y=[row['champion']],
                name='Decrease',
                orientation='h',
                marker_color='red',
                base=row['winrate_w12'],
                hovertemplate='%{x}%'
            ))

    # Add a scatter trace for the y-axis line
    fig.add_trace(go.Scatter(
        x=[0, 0],
        y=[df['champion'].min(), df['champion'].max()],
        mode='lines',
        line=dict(color='black', width=2),
        showlegend=False
    ))

    # Set layout
    fig.update_layout(
        barmode='overlay',
        title=f'Win Rate Comparison for {role.capitalize()} Role',
        xaxis=dict(
            title='Win Rate (%)',
            zeroline=True,
            zerolinewidth=2,
            zerolinecolor='black',
            range=[-100, 100],
        ),
        yaxis=dict(
            title='Champion',
            showline=True,
            linecolor='black',
            linewidth=2,
            layer='above traces'
        ),
        legend=dict(
            font=dict(
                color='black'
            )
        ),
        showlegend = False,
        hoverlabel=dict(
            font_size=16,
            font_color='black'
        )
    )
    return fig

# Example usage for 'top' role
fig_top = create_horizontal_bar_chart(df_combined, 'top')
fig_top.show()

# Example usage for 'jungle' role
fig_jungle = create_horizontal_bar_chart(df_combined, 'jungle')
fig_jungle.show()